# Py07 - Working with Fabrics
**This tutorial is designed for developers working with the pyc8 Python driver**

Each :doc:`tenant <tenant>` on the C8 Data Fabric server can have an arbitrary number of fabrics. Each fabric has its own set of :doc:`collections <collection>`, :doc:`graphs <graph>` and :doc:`streams <stream>`.

For each tenant, there is a special fabric named _system, which cannot be dropped and provides operations for managing users, permissions and other fabrics. Most of the operations can only be executed by admin users. See :doc:`user` for more information.

Each fabric in the C8 Fabric can be replicated to one or more additional edge Locations in the fabric. If a change is made to such a replicated fabric in one edge Location, that change will be automatically propagated to, and visible in, all other Edge Locations to which that fabric has been replicated.

Each fabric in the C8 Fabric can publish changes in realtime to any clients which are connected to that fabric. Any clients with connections to that fabric will receive changes via a push-based mechanism rather than having to continuously poll the fabric for any changes which may have occurred. This python driver can listen in realtime to changes in fabric by calling the fabric.on_change() method for the fabric referred to by the fabric object.

## 1. Import Libraries to Workbook

In [ ]:
import json
from c8 import C8Client

## 2. Create and Define Login details

In [ ]:
fed_url = "gdn.paas.macrometa.io"
admin_email = "email"
admin_password = "password"
geo_fabric = "_system"

## 3. Connect to GDN as Admin

In [ ]:
# Initialize the C8 Data Fabric client.
print("\n ------- CONNECTION SETUP  ------")
print("tenant: {}, geofabric:{}".format(admin_email, geo_fabric))
client = C8Client(protocol='https', host=fed_url, port=443,
                email=admin_email, password=admin_password,
                geofabric=geo_fabric)  

# For the "mytenant" tenant, connect to "_system" fabric as tenant admin.
# This returns an API wrapper for the "_system" fabric on tenant 'mytenant'
# Note that the 'mytenant' tenant should already exist.

tenant = client.tenant(email=admin_email, password=admin_password)

sys_fabric = tenant.useFabric('_system')

## 4. List all Fabrics accessable by Admin

At this point in the tutorial this should only show the defualt 'system' fabric which will have all the avaiailble datacenters

In [ ]:
# List all fabrics in the 'mytenant' tenant
response = sys_fabric.fabrics()
print(response)

## 5. List all details of Datacenters

Now lets have a look at those data centers

In [ ]:
#Returns the list of details of Datacenters
response = sys_fabric.dclist(detail=False) # Tip try changing details to = "True" to see more details..

print(json.dumps(response, indent=4)) 

## 6. Create a new user called "johndoe"

In [ ]:
# Create a new user.
tenant.create_user(
    username='johndoe', 
    email='johndoe@gmail.com', 
    password='first_password',
    active=True,
    extra={'team': 'backend', 'title': 'engineer'}
)

## 7. List details of the new user

In [ ]:
# Retrieve details of a user.
tenant.user('johndoe')

## 8. List permissions of the new user

Lets take a look at the permission of our newly created user, at this point they should be read only or 'ro'

In [ ]:
# Retrieve user permissions for all fabrics and collections.

tenant.permissions('johndoe')


## 9. Create a new Fabric

While we are still connected as the Admin, we can create a new Fabric. 

The code block below checks to see what the local datacenter is to you using the "localdc()" method.

In [ ]:
mylocal_dc_response = sys_fabric.localdc() #<--- Returns the datacenter thats closest to you

mylocal_dc_name = mylocal_dc_response['_key'] #<--- Selects the name of that datacenter

dcl = [mylocal_dc_name, 'gdn-us-west', 'gdn-eu-west'] #<--- Create a DataCenter List for the New GeoFabric (Macrometa GDN)

if not sys_fabric.has_fabric('demofabric'): #<--- checks the geofabric doesnt already exist
    demo_fabric = sys_fabric.create_fabric('demofabric', dclist=dcl) #<--- create a geofabric
    

all_fabric_details = sys_fabric.fabrics_detail()  #<--- get details of new geofabric

print(json.dumps(all_fabric_details, indent=4)) 

## 10. Check permissions of new user on new fabric

We have created a new user, and a new geofabric. Wjhen this is first created the user should have no permissions.

In [ ]:
# Retrieve user permission for "test" fabric.
tenant.permission(
    username='johndoe',
    fabric='demofabric'
)

## 11. Change permissions of new user to "read/write"

Lets change our new users permissions to "read/right" so when we connect at this user they can work with the database and create a collection

In [ ]:
# Update user permission for "test" fabric.
tenant.update_permission(
    username='johndoe',
    permission='rw',
    fabric='demofabric'
)

## 12. Change permissions of new user to "none" for the "_system" fabric 

We might also want to reduce the users access to the _system fabric, we can do this as follows...

In [ ]:
# Update user permission for "_system" fabric.
tenant.update_permission(
    username='johndoe',
    permission='none',
    fabric='_system'
)

## 13. Check permissions of new user after changes

Ok, so now that we have updated our users permissions, lets take a look to see our changes

In [ ]:
# Retrieve user permissions for all fabrics and collections.

tenant.permissions('johndoe')

## 14. Connect to GDN as the new user "JohnDoe" on the new GeoFabric

Right, now its time to connect to our new GeoFabric as our new user.

In [ ]:
demo_email = 'johndoe@gmail.com'
demo_password = 'first_password'
new_geofabric = 'demofabric'

local_connected_fed = mylocal_dc_name + '.paas.macrometa.io' # using Macrometa Global PaaS

demo_fabric_client = C8Client(protocol='https', 
                              host=local_connected_fed,
                              port=443,
                              email=demo_email,
                              password=demo_password,
                              geofabric=new_geofabric
                             )  

demo_tenant = demo_fabric_client.tenant(email=demo_email, password=demo_password)

demo_fabric = tenant.useFabric(new_geofabric)

print("\n ------- CONNECTION SETUP  ------")
print("tenant: {}, geofabric: {}".format(demo_email, new_geofabric))

## 15. Check the Datacenters for the new geofabric

Now that we have connected as our new user, lets see what datacenters we have access to in our geofabric

In [ ]:
response = demo_fabric.dclist(detail=False) # Tip try changing details to = "True" to see more details..

print(json.dumps(response, indent=4)) 

## 16. Let's test our new fabric with a collection

### 16.1 First create the collection

You will likely remember this step from an earlier tutorial. First we define a collection name, then we check to see if it exists. If it doesnt exist then we will create it.

In [ ]:
collection_name = 'employees'

# Create a new collection if it does not exist
if demo_fabric_client.has_collection(collection_name):
    print("Collection exists")
else:
    demo_fabric_client.create_collection(name=collection_name)
    print("Collection Created!")

### 16.2 Let's add some records

A collection is great, but lets add some records to it..

In [ ]:
docs = [
  {'_key':'James', 'firstname': 'James', 'lastname':'Kirk', 'email':'[email protected]'},
  {'_key':'Han', 'firstname': 'Han', 'lastname':'Solo', 'email':'[email protected]'},
  {'_key':'Bruce', 'firstname': 'Bruce', 'lastname':'Wayne', 'email':'[email protected]'}
]

demo_fabric_client.insert_document(collection_name='employees', document=docs)

print(json.dumps(docs, indent=4))

### 16.3 Let's build a RESTQL called "getRecords"

You likely recall how to create a RESTQL from the earlier tutorial. So lets set up a basic query to show the records in a collection

In [ ]:
# crete restql
get_data = {"query": {"name": "getRecords", "value": "FOR doc IN %s RETURN doc" % collection_name}}

demo_fabric_client.create_restql(get_data)

### 16.4 Let's check our RESTQL is saved

In [ ]:
# get all restql
response = demo_fabric_client.get_restqls()
    
print(json.dumps(response, indent=4)) 

### 16.5 Let's execute the RESTQL "getRecords"

In [ ]:
# execute restql without bindVars
test_response = demo_fabric_client.execute_restql("getRecords")

print(json.dumps(test_response, indent=4)) 

## 17. Time to tidy up!

That was great!, now lets tidy up by removing our fabric, the new user, and the collection we created.

In [ ]:
# delete restql
response = demo_fabric_client.delete_restql("getRecords")
print(response) 

In [ ]:
#delete new user
tenant.delete_user(username='johndoe')

# delete the fabric

In [ ]:
# first connect to the _system fabric

sys_fabric = tenant.useFabric('_system')

# now delete the demo geo fabric we created earlier
sys_fabric.delete_fabric('demofabric')
#print(response) 

## Section Completed!

Congratulations!, another tutorial completed.